In [2]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.8 MB/s eta 0:00:00


In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...


In [4]:
!conda install -c bioconda seqkit

Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - seqkit


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    openssl-3.3.0              |       hd590300_0         2.8 MB  conda-forge
    seqkit-2.8.1               |       h9ee0642_0         7.1 MB  bioconda
    ------------------------------------------------------------
                                           Total:        10.2 MB

The following NEW packages wil

In [81]:
import subprocess
import os
from Bio import SeqIO
import requests
import re
import json


def seqkit_stats(fasta_file):
    """
    Call "seqkit stats" on a given fasta file.

    Args:
        fasta_file (str): Path to the fasta file.

    Returns:
        str: Output from seqkit stats.
    """
    try:
        output = subprocess.check_output(["seqkit", "stats", fasta_file])
        return output.decode("utf-8")
    except subprocess.CalledProcessError as e:
        print(str(e))
        return str(e)


def parse_seqkit_output(output):
    """
    Parse an output from seqkit.

    Args:
        output (str): Output from seqkit stats.

    Returns:
        tuple: File type (DNA or Protein), output from seqkit stats.
    """
    try:
        lines = output.split("\n")
        return lines[1].split()[2]
    except Exception as e:
        print(str(e))
        return None, str(e)


def save_stats(file_type, output):
    """
    Save the stats output to a file.

    Args:
        file_type (str): File type (DNA or Protein).
        output (str): Output from seqkit stats.
    """
    global file
    file = f"{file_type}_stats.txt"
    with open(file, "w") as f:
        f.write(output)


def read_fasta_with_biopython(fasta_file):
    """
    Read a given fasta file with biopython.

    Args:
        fasta_file (str): Path to the fasta file.

    Returns:
        list: List of sequences.
    """
    sequences = list(SeqIO.parse(fasta_file, "fasta"))
    return sequences


def iterate_over_sequences(sequences, file_type):
    """
    Iterate over sequences to read the descriptions and search for IDs.

    Args:
        sequences (list): List of sequences.
        file_type (str): File type (DNA or Protein).
    """
    matched_ids = []
    reads = {}
    descriptions = {}

    for sequence in sequences:
        description = sequence.description

        # Search for Uniprot ID
        if file_type == "Protein":
            uniprot_id = re.search(r'[OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9]([A-Z][A-Z0-9]{2}[0-9]){1,2}', description)
            if uniprot_id:
                matched_ids.append(uniprot_id.group(0))
                reads[uniprot_id.group(0)] = sequence.seq
                descriptions[uniprot_id.group(0)] = sequence.description

        # Search for ENSEMBL ID
        elif file_type == "DNA":
            ensembl_id = re.search(r'(ENS([A-Z]{0,4})|(MGP\D*))(E|FM|G|GT|P|RF|T)\d{11}', description)
            if ensembl_id:
                matched_ids.append(ensembl_id.group(0))
                reads[ensembl_id.group(0)] = sequence.seq
                descriptions[ensembl_id.group(0)] = sequence.description

    return matched_ids, reads, descriptions

In [83]:
def get_uniprot(ids: list):
    accessions = ','.join(ids)
    endpoint = "https://rest.uniprot.org/uniprotkb/accessions"
    http_function = requests.get
    http_args = {'params': {'accessions': accessions}}
    return http_function(endpoint, **http_args)


def get_ensembl(accessions_list):
    json_ids = json.dumps({"ids": accessions_list})
    server = "https://rest.ensembl.org"
    ext = "/lookup/id"
    headers = {"Content-Type": "application/json", "Accept": "application/json"}
    r = requests.post(server + ext, headers=headers, data=f'{json_ids}')
    if not r.ok:
        raise ValueError('Incorrect data entered.')

    decoded = r.json()
    return decoded

def parse_response_uniprot(resp = None):
    resp = resp.json()
    resp = resp["results"]
    output = {}
    for val in resp:
        acc = val['primaryAccession']
        species = val['organism']['scientificName']
        gene = val['genes']
        seq = val['sequence']
        output[acc] = {'organism':species, 'geneInfo':gene, 'sequenceInfo':seq, 'type':'protein'}
    if file_type[0]!=None:
      with open(f"{file_type}_stats.txt", "a") as f:
          f.write('---------------------------------------\nInfo from Uniprot\n(Name, read sequence, Uniprot info, description)\n\n')
          for keys,values in output.items():
            f.write(str(keys)+'\n')
            f.write(str(reads[keys])+'\n')
            f.write(str(values)+'\n')
            f.write(str(descriptions[keys])+'\n\n')
      return output
    else:
      print('Something`s wrong with your fasta file!')


def parse_response_ensembl(decoded):
    output = {}
    for val in decoded:
        acc = decoded[val]['id']
        species = decoded[val]['species']
        type = decoded[val]["object_type"]
        logic_name = decoded[val]['logic_name']
        output[acc] = {'organism':species, 'logic_name':logic_name, 'type':type}
    if file_type[0]!=None:
      with open(f"{file_type}_stats.txt", "a") as f:
          f.write('---------------------------------------\nInfo from Ensembl\n(Name, read sequence, Ensembl info, description)\n\n')
          for keys,values in output.items():
            f.write(str(keys)+'\n')
            f.write(str(reads[keys])+'\n')
            f.write(str(values)+'\n')
            f.write(str(descriptions[keys])+'\n\n')
      return output
    else:
      print('Something`s wrong with your fasta file!')

def parse_ID(ID_list):
    ensembl_match = all(re.search(r'(ENS([A-Z]{0,4})|(MGP\D*))(E|FM|G|GT|P|RF|T)\d{11}', 'r' + ID) for ID in ID_list)
    uniprot_match = all(re.search(r'[OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9]([A-Z][A-Z0-9]{2}[0-9]){1,2}', 'r' + ID) for ID in ID_list)

    if ensembl_match:
        return parse_response_ensembl(get_ensembl(ID_list))
    elif uniprot_match:
        return parse_response_uniprot(get_uniprot(ID_list))
    else:
        return 'Check the ids\' format'


In [87]:
if __name__ == "__main__":

    fasta_file = str(input('What is your fasta file path?\n'))
    #with open(fasta_file_path) as fasta_file:

    # Call seqkit stats
    output = seqkit_stats(fasta_file)

    # Parse the output
    file_type = parse_seqkit_output(output)

    # Save the stats
    if file_type is not None:
        save_stats(file_type, output)

    # Read the fasta file with biopython
    sequences = read_fasta_with_biopython(fasta_file)

    # Iterate over sequences to read the descriptions and search for IDs
    matched_ids, reads, descriptions = iterate_over_sequences(sequences, file_type)

    # Perform an API call to a given database (Uniprot or ENSEMBL)
    #api_call(matched_ids, file_type)

    print(parse_ID(matched_ids))

What is your fasta file path?
/content/hw_file3.fasta
list index out of range
Something`s wrong with your fasta file!
None
